In [2]:
import csv
import cv2
import pytesseract 
from pytesseract import image_to_string
import pandas as pd
import re
import os
import numpy as np
from moviepy.video.io.VideoFileClip import VideoFileClip
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
%run helper-extractingVideos.ipynb

In [3]:
def extract_from_video(filePath,videoName,template,templateRes):
        # Open the video using OpenCV
    
    desired_width = 1631
    desired_height = 907
    cap = cv2.VideoCapture(filePath)
    #,apiPreference=cv2.CAP_ANY,params=[
    #cv2.CAP_PROP_FRAME_WIDTH,desired_width ,
    #cv2.CAP_PROP_FRAME_HEIGHT, desired_height])
    
    #cap.set(cv2.CAP_PROP_FRAME_WIDTH, desired_width)
    #cap.set(cv2.CAP_PROP_FRAME_HEIGHT, desired_height)
    fps=cap.get(cv2.CAP_PROP_FPS)
    print("FPS:",fps)
    minToSkip=11
    framesToSkip=fps*60*minToSkip
    framesToSkipEverySec=fps-1
    currentFrame=0
    currentFrameInSec=0
    jumpStarted=False
    sno=1
    row={}#changed

    # Initialize a list to store the scores

    print("start")

    oldText="old"
    # Process each frame of the video
    while True:
        # Read the next frame
        ret, frame = cap.read()
        currentFrame+=1
        currentFrameInSec+=1
        if currentFrame<framesToSkip :
            if currentFrame%(fps*60)==0:
                print(currentFrame//(fps*60) ," min(s) Skipped")
            continue
        if currentFrame//(fps*60)<minToSkip:
            continue
        if currentFrameInSec<framesToSkipEverySec:
            continue



        # Break the loop if the video has ended
        if not ret:
            print("completed")
            break

        current_pos = cap.get(cv2.CAP_PROP_POS_MSEC)
        total_seconds = int(current_pos // 1000)
        minutes = total_seconds // 60
        seconds = total_seconds % 60
        timeInVideo =f"{minutes} minutes and {seconds} seconds"
        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        text = image_to_string(gray,lang="eng")   

        words = text.lower().split()  # split the text into a list of words
        filtered_words = [word for word in words if (word != 'olympic' and word != 'olympic' and word!='2020' and word!='202' and word!='tokyo' and word!='tokyo2020' and word!="tok" and word!="toky") ]  # use list comprehension to remove the word "hello"
        text = ' '.join(filtered_words)  # join the remaining words back into a string
    
        
        if not jumpStarted and 'attempt' in text.lower(): #and extract_frame(frame,template):
            jumpStarted = True
            row['Time'] = timeInVideo
            print(timeInVideo," jump started")
            if 'Time' in row:  # Check if 'Time' key exists in row
                # Extract minutes and seconds from time duration string                    time_str = row['Time']
                time_str = row['Time']
                minutes_str, seconds_str = re.match(r'(\d+) minutes and (\d+) seconds', time_str).groups()
                # Convert minutes and seconds to integers
                minutes = int(minutes_str)
                seconds = int(seconds_str)
                # Calculate total seconds
                totalSeconds = minutes * 60 + seconds            

      
        if jumpStarted and 'attempt' not in text.lower() and ('1.84' in text.lower() or '1.84m' in text.lower() ): #and extract_frame(frame,templateRes):
            jumpStarted = False
            print(timeInVideo," jump scores announced")
            #plt.imshow(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
            extract_clip(filePath,'ExtractedVideos/'+str(sno)+'_'+videoName+'.mp4',totalSeconds,int(total_seconds))
            sno+=1                #print(text)
            print(row)
                #save_data(row,"scores")

        

        currentFrameInSec=0    
    cap.release()

In [ ]:
templateLondon = cv2.imread('helperImages/template1.jpg', cv2.IMREAD_GRAYSCALE)
templateResLondon = cv2.imread('helperImages/template2.jpg', cv2.IMREAD_GRAYSCALE)
extract_from_video("video/Women's_High_Jump_Final_Tokyo_Replays.mp4","Women's_High_Jump_Final_Tokyo_Replays",templateLondon,templateResLondon)

FPS: 29.97002997002997
start
11 minutes and 12 seconds  jump started
11 minutes and 33 seconds  jump scores announced
video/Women's_High_Jump_Final_Tokyo_Replays.mp4
ExtractedVideos/1_Women's_High_Jump_Final_Tokyo_Replays.mp4
672
693
Moviepy - Building video ExtractedVideos/1_Women's_High_Jump_Final_Tokyo_Replays.mp4.
MoviePy - Writing audio in 1_Women's_High_Jump_Final_Tokyo_ReplaysTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ExtractedVideos/1_Women's_High_Jump_Final_Tokyo_Replays.mp4



Moviepy - Done !
Moviepy - video ready ExtractedVideos/1_Women's_High_Jump_Final_Tokyo_Replays.mp4
{'Time': '11 minutes and 12 seconds'}
11 minutes and 49 seconds  jump started
12 minutes and 26 seconds  jump scores announced
video/Women's_High_Jump_Final_Tokyo_Replays.mp4
ExtractedVideos/2_Women's_High_Jump_Final_Tokyo_Replays.mp4
709
746
Moviepy - Building video ExtractedVideos/2_Women's_High_Jump_Final_Tokyo_Replays.mp4.
MoviePy - Writing audio in 2_Women's_High_Jump_Final_Tokyo_ReplaysTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ExtractedVideos/2_Women's_High_Jump_Final_Tokyo_Replays.mp4



Moviepy - Done !
Moviepy - video ready ExtractedVideos/2_Women's_High_Jump_Final_Tokyo_Replays.mp4
{'Time': '11 minutes and 49 seconds'}
12 minutes and 34 seconds  jump started
13 minutes and 56 seconds  jump scores announced
video/Women's_High_Jump_Final_Tokyo_Replays.mp4
ExtractedVideos/3_Women's_High_Jump_Final_Tokyo_Replays.mp4
754
836
Moviepy - Building video ExtractedVideos/3_Women's_High_Jump_Final_Tokyo_Replays.mp4.
MoviePy - Writing audio in 3_Women's_High_Jump_Final_Tokyo_ReplaysTEMP_MPY_wvf_snd.mp3


...

Moviepy - Done !
Moviepy - video ready ExtractedVideos/7_Women's_High_Jump_Final_Tokyo_Replays.mp4
{'Time': '16 minutes and 50 seconds'}
16 minutes and 53 seconds  jump started
17 minutes and 23 seconds  jump scores announced
video/Women's_High_Jump_Final_Tokyo_Replays.mp4
ExtractedVideos/8_Women's_High_Jump_Final_Tokyo_Replays.mp4
1013
1043
Moviepy - Building video ExtractedVideos/8_Women's_High_Jump_Final_Tokyo_Replays.mp4.
MoviePy - Writing audio in 8_Women's_High_Jump_Final_Tokyo_ReplaysTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ExtractedVideos/8_Women's_High_Jump_Final_Tokyo_Replays.mp4



Moviepy - Done !
Moviepy - video ready ExtractedVideos/8_Women's_High_Jump_Final_Tokyo_Replays.mp4
{'Time': '16 minutes and 53 seconds'}
17 minutes and 33 seconds  jump started
18 minutes and 8 seconds  jump scores announced
video/Women's_High_Jump_Final_Tokyo_Replays.mp4
ExtractedVideos/9_Women's_High_Jump_Final_Tokyo_Replays.mp4
1053
1088
Moviepy - Building video ExtractedVideos/9_Women's_High_Jump_Final_Tokyo_Replays.mp4.
MoviePy - Writing audio in 9_Women's_High_Jump_Final_Tokyo_ReplaysTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ExtractedVideos/9_Women's_High_Jump_Final_Tokyo_Replays.mp4



Moviepy - Done !
Moviepy - video ready ExtractedVideos/9_Women's_High_Jump_Final_Tokyo_Replays.mp4
{'Time': '17 minutes and 33 seconds'}
